In [345]:
import pandas as pd

In [346]:
import numpy as np

In [347]:
import copy

In [348]:
from sklearn.model_selection import train_test_split

In [349]:
from sklearn.linear_model import LinearRegression

In [350]:
from sklearn.ensemble import RandomForestRegressor

In [351]:
from sklearn.metrics import mean_squared_error

In [352]:
from sklearn import metrics

In [353]:
df0 = pd.read_csv('train.csv')

In [354]:
df0

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,1456,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,1457,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,1458,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,1459,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [355]:
def preprocessing(df):
    df['total_rooms'] = df['TotRmsAbvGrd'] + df['BsmtFullBath']
    df['price_by_building'] = df['SalePrice'] / (df['GrLivArea'] + df['GarageArea'] + df['TotalBsmtSF'])
    df['qual_area_pound'] = df['OverallQual'] * df['GrLivArea']
    
    df.drop_duplicates()
#     drop outsider qual area pound
    df = df.drop(523)
    df = df.drop(1298)
#     drop outsider price by building
    df = df.drop(533)
#     create a bool columns for yearbuilt
    df['built_after_85'] = df['YearBuilt'] > 1985
    df = df[['total_rooms', 'price_by_building', 'TotalBsmtSF', 'GarageArea', 'qual_area_pound', 'SalePrice']]
    return df
df = df0.copy()
df = preprocessing(df)
df

,total_rooms,price_by_building,TotalBsmtSF,GarageArea,qual_area_pound,SalePrice
0,9,66.955684,856,548,11970,208500
1,6,60.824397,1262,460,7572,181500
2,7,67.441159,920,608,12502,223500
3,8,44.943820,756,642,12019,140000
4,10,59.822924,1145,836,17584,250000
...,...,...,...,...,...,...
1455,7,57.189542,953,460,9882,175000
1456,8,51.032807,1542,500,12438,210000
1457,9,71.180556,1152,252,16380,266500
1458,6,59.317613,1078,240,5390,142125


In [356]:
def normalise (data):
    for column in data.columns:
        data[column] = data[column] / data[column].mean()
    return data

In [357]:
X, y = df.drop(columns="SalePrice"), df["SalePrice"]
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [358]:
reg = LinearRegression().fit(X_train, y_train)
reg.score(X_train, y_train)

0.9685791864774614

In [359]:
reg.score(X_test, y_test)

0.9672551041292626

In [360]:
reg.coef_

array([3673.8845229 , 2785.89604994,   58.30661724,   51.75758903,
          6.30891052])

In [361]:
for name, feature in zip(['total_rooms',  'price_by_building', 'TotalBsmtSF', 'GarageArea', 'qual_area_pound'], reg.coef_):
    print('coef', name,' : ', feature)

coef total_rooms  :  3673.884522899027
coef price_by_building  :  2785.8960499385244
coef TotalBsmtSF  :  58.30661723705772
coef GarageArea  :  51.75758903172985
coef qual_area_pound  :  6.308910524453792


In [362]:
reg.intercept_

-155147.55046558226

In [363]:
df_mean = normalise(copy.deepcopy(df))

In [364]:
df_mean

,total_rooms,price_by_building,TotalBsmtSF,GarageArea,qual_area_pound,SalePrice
0,1.297161,1.139474,0.812715,1.160098,1.243917,1.151742
1,0.864774,1.035129,1.198185,0.973805,0.786879,1.002596
2,1.008903,1.147736,0.873479,1.287116,1.299202,1.234601
3,1.153032,0.764869,0.717772,1.359092,1.249009,0.773352
4,1.441290,1.018086,1.087102,1.769784,1.827321,1.380986
...,...,...,...,...,...,...
1455,1.008903,0.973270,0.904810,0.973805,1.026933,0.966690
1456,1.153032,0.868493,1.464027,1.058483,1.292551,1.160028
1457,1.297161,1.211374,1.093748,0.533476,1.702202,1.472131
1458,0.864774,1.009487,1.023490,0.508072,0.560126,0.785090


In [365]:
X_mean, y_mean = df_mean.drop(columns="SalePrice"), df_mean["SalePrice"]
X_mean_train, X_mean_test, y_mean_train, y_mean_test = train_test_split(X_mean, y_mean)

In [366]:
reg_mean = LinearRegression().fit(X_mean_train, y_mean_train)
reg_mean.score(X_mean_train, y_mean_train)

0.9659659343878917

In [367]:
reg_mean.score(X_mean_test, y_mean_test)

0.9771944717204077

In [368]:
reg_mean.coef_

array([0.10723225, 0.89361224, 0.33452649, 0.12635525, 0.37523232])

In [369]:
for name, feature in zip(['total_rooms',  'price_by_building', 'TotalBsmtSF', 'GarageArea', 'qual_area_pound'], reg_mean.coef_):
    print('coef', name,' : ', feature)

coef total_rooms  :  0.10723224977459624
coef price_by_building  :  0.8936122358119708
coef TotalBsmtSF  :  0.334526491277933
coef GarageArea  :  0.1263552505289664
coef qual_area_pound  :  0.37523231513943345


# RANDOM FOREST

In [370]:
X, y = df.drop(columns="SalePrice"), df["SalePrice"]
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [371]:
regr = RandomForestRegressor(max_depth=500, random_state=0)

In [372]:
regr_fit = regr.fit(X_train, y_train)

In [373]:
y_pred = regr_fit.predict(X_test)

In [374]:
mean_squared_error(y_pred, y_test)

126281375.95643124

In [375]:
np.sqrt(mean_squared_error(y_pred, y_test))

11237.498652121443

In [376]:
y_test.mean()

179567.38356164383

In [377]:
mean_squared_error(y_pred, y_test) / y_test.mean()**2

0.003916376139199601

In [378]:
print('Mean Absolute Error:', metrics.mean_absolute_error(y_test, y_pred))  

Mean Absolute Error: 7981.308219178082


In [379]:
importances_features = regr_fit.feature_importances_

In [380]:
for name, feature in zip(['total_rooms',  'price_by_building', 'TotalBsmtSF', 'GarageArea', 'qual_area_pound'], importances_features):
    print(name, round(feature*100,1),'%')

total_rooms 0.4 %
price_by_building 25.5 %
TotalBsmtSF 8.2 %
GarageArea 1.4 %
qual_area_pound 64.6 %


# RANDOM FOREST NORMALISE

In [381]:
X_mean, y_mean = df_mean.drop(columns="SalePrice"), df_mean["SalePrice"]
X_mean_train, X_mean_test, y_mean_train, y_mean_test = train_test_split(X_mean, y_mean)

In [382]:
regr_mean = RandomForestRegressor(max_depth=5, random_state=0)

In [383]:
regr_mean_fit = regr.fit(X_mean_train, y_mean_train)

In [384]:
y_mean_pred = regr_mean_fit.predict(X_mean_test)

In [385]:
mean_squared_error(y_mean_pred, y_mean_test)

0.007100904030983451

In [336]:
importances_features_mean = regr_mean_fit.feature_importances_

In [337]:
for name, feature in zip(['total_rooms',  'price_by_building', 'TotalBsmtSF', 'GarageArea', 'qual_area_pound'], importances_features_mean):
    print(name, round(feature*100,1),'%')

total_rooms 0.4 %
price_by_building 24.7 %
TotalBsmtSF 7.4 %
GarageArea 1.5 %
qual_area_pound 66.0 %


## TEST NORMALISE

In [439]:
test = X_mean_test[:1]

In [434]:
prediction = regr_mean_fit.predict(test)

In [435]:
prediction

array([0.96186697])

In [436]:
test

,total_rooms,price_by_building,TotalBsmtSF,GarageArea,qual_area_pound
1430,1.008903,1.111455,0.694985,0.787511,0.955021


In [458]:
df_mean.loc[(df_mean['GarageArea'] == 0.787511)]

,total_rooms,price_by_building,TotalBsmtSF,GarageArea,qual_area_pound,SalePrice


## NON NORMALISE

In [449]:
no_normalise_test = X_test[5:6]

In [450]:
no_normalise_test

,total_rooms,price_by_building,TotalBsmtSF,GarageArea,qual_area_pound
439,7,45.681063,684,528,7176


In [460]:
regr_fit = regr.fit(X_train, y_train)

In [461]:
prediction_non_normalise = regr_fit.predict(no_normalise_test)

In [462]:
prediction_non_normalise

array([122109.])

In [463]:
df.loc[(df['GarageArea'] == 528) & (df['TotalBsmtSF'] ==684)]

,total_rooms,price_by_building,TotalBsmtSF,GarageArea,qual_area_pound,SalePrice
439,7,45.681063,684,528,7176,110000


In [465]:
no_normalise_test = X_test[24:25]

In [466]:
no_normalise_test

,total_rooms,price_by_building,TotalBsmtSF,GarageArea,qual_area_pound
786,8,50.472041,686,416,8260


In [467]:
prediction_non_normalise = regr_fit.predict(no_normalise_test)

In [468]:
prediction_non_normalise

array([133758.64])

In [469]:
df.loc[(df['GarageArea'] == 416) & (df['TotalBsmtSF'] ==686)]

,total_rooms,price_by_building,TotalBsmtSF,GarageArea,qual_area_pound,SalePrice
786,8,50.472041,686,416,8260,139000
